In [24]:
from keras.utils.vis_utils import plot_model
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, LSTM, TimeDistributed, Dense, RepeatVector, CuDNNLSTM, GRU, Bidirectional, Input, CuDNNGRU
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import tensorflow as tf
import os
from keras import backend as K
from keras.models import Model
from keras.layers.core import Dense, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers import concatenate
import difflib
from keras.models import load_model
import keras
from keras import losses
import matplotlib.pyplot as plt
import random
from random import choice
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from keras_transformer import get_model, decode

train_index = 80000
valid_index = 90000
test_index = 100000
# X_train=np.load('prepData/X_train_camFam3_v3_chr2_size11.npy')
# X_val=np.load('prepData/X_val_camFam3_v3_chr2_size11.npy')
# X_test=np.load('prepData/X_test_camFam3_v3_chr2_size11.npy')
# y_train=np.load('prepData/y_train_camFam3_v3_chr2_size11.npy')
# y_val=np.load('prepData/y_val_camFam3_v3_chr2_size11.npy')
# y_test=np.load('prepData/y_test_camFam3_v3_chr2_size11.npy')

# y_train1 = np.load('prepData/y_train1_camFam3_v3_chr2_size11.npy')
# y_val1 = np.load('prepData/y_val1_camFam3_v3_chr2_size11.npy')
# y_test1 = np.load('prepData/y_test1_camFam3_v3_chr2_size11.npy')

In [25]:
os.environ["CUDA_VISIBLE_DEVICES"]="6"
with open ('ancestral', 'rb') as fp:
    source_tokens = pickle.load(fp)
with open ('descendant', 'rb') as f:
    target_tokens = pickle.load(f)

print(len(source_tokens))
source_tokens = source_tokens[:100000]
target_tokens = target_tokens[:100000]






1242976


In [26]:
# Generate dictionaries
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict

In [27]:
print(source_tokens[1])
print(target_tokens[1])

['A', 'G', 'A', 'C', 'A', 'G', '-', 'A', 'A', 'A', 'C']
['A', 'G', 'A', 'G', 'A', 'G', '-', 'A', 'A', 'A', 'C']


In [28]:
source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}

# Add special tokens
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]

# Padding
source_max_len = max(map(len, encode_tokens))
target_max_len = max(map(len, decode_tokens))

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

In [29]:
# Build & fit model
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=True,  # Use different embeddings for different languages
)
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

model.fit(
    x=[np.array(encode_input[:train_index]), np.array(decode_input[:train_index])],
    y=np.array(decode_output[:train_index]),
    epochs=200,
    batch_size=1000,
    validation_data=([np.array(encode_input[train_index:valid_index]), np.array(decode_input[train_index:valid_index])], np.array(decode_output[train_index:valid_index]))
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 32), ( 384         Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 32)     0           Token-Embedding[0][0]            
__________

Train on 80000 samples, validate on 10000 samples
Epoch 1/200
80000/80000 [==============================] - 12s 151us/step - loss: 1.7919 - val_loss: 1.3133
Epoch 2/200
80000/80000 [==============================] - 4s 51us/step - loss: 1.1177 - val_loss: 1.0231
Epoch 3/200
80000/80000 [==============================] - 4s 56us/step - loss: 0.8966 - val_loss: 0.7142
Epoch 4/200
80000/80000 [==============================] - 5s 58us/step - loss: 0.4759 - val_loss: 0.4805
Epoch 5/200
80000/80000 [==============================] - 4s 56us/step - loss: 0.3785 - val_loss: 0.4653
Epoch 6/200
80000/80000 [==============================] - 4s 55us/step - loss: 0.3653 - val_loss: 0.4618
Epoch 7/200
80000/80000 [==============================] - 4s 53us/step - loss: 0.3598 - val_loss: 0.4584
Epoch 8/200
80000/80000 [==============================] - 4s 50us/step - loss: 0.3568 - val_loss: 0.4569
Epoch 9/200
80000/80000 [==============================] - 4s 50us/step - loss: 0.3546 - val_loss: 0

80000/80000 [==============================] - 4s 53us/step - loss: 0.3272 - val_loss: 0.4579
Epoch 78/200
80000/80000 [==============================] - 4s 54us/step - loss: 0.3268 - val_loss: 0.4553
Epoch 79/200
80000/80000 [==============================] - 4s 51us/step - loss: 0.3265 - val_loss: 0.4580
Epoch 80/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.3261 - val_loss: 0.4547
Epoch 81/200
80000/80000 [==============================] - 4s 53us/step - loss: 0.3260 - val_loss: 0.4561
Epoch 82/200
80000/80000 [==============================] - 4s 54us/step - loss: 0.3254 - val_loss: 0.4578
Epoch 83/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.3252 - val_loss: 0.4545
Epoch 84/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.3247 - val_loss: 0.4574
Epoch 85/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.3245 - val_loss: 0.4579
Epoch 86/200
80000/80000 [========================

80000/80000 [==============================] - 4s 54us/step - loss: 0.2701 - val_loss: 0.4469
Epoch 154/200
80000/80000 [==============================] - 4s 53us/step - loss: 0.2688 - val_loss: 0.4513
Epoch 155/200
80000/80000 [==============================] - 4s 53us/step - loss: 0.2681 - val_loss: 0.4492
Epoch 156/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.2680 - val_loss: 0.4479
Epoch 157/200
80000/80000 [==============================] - 4s 54us/step - loss: 0.2672 - val_loss: 0.4524
Epoch 158/200
80000/80000 [==============================] - 4s 52us/step - loss: 0.2667 - val_loss: 0.4515
Epoch 159/200
80000/80000 [==============================] - 5s 58us/step - loss: 0.2663 - val_loss: 0.4480
Epoch 160/200
80000/80000 [==============================] - 4s 53us/step - loss: 0.2657 - val_loss: 0.4511
Epoch 161/200
80000/80000 [==============================] - 4s 51us/step - loss: 0.2652 - val_loss: 0.4437
Epoch 162/200
80000/80000 [===============

In [30]:
# Predict
def inference(test):
    result = []
    for i in range(len(test)):
        output = decode(
            model,
            test[i],
            start_token=target_token_dict['<START>'],
            end_token=target_token_dict['<END>'],
            pad_token=target_token_dict['<PAD>'],
        )
        #print(output)
        result.append(''.join(map(lambda x: target_token_dict_inv[x], output[1:-1])))
    return result

In [31]:
decoded = inference(encode_input[90000:91000])
for i in range(100):
    print(''.join(source_tokens[90000+i]), '->', decoded[i], ''.join(target_tokens[90000+i]))

TA--------- -> TAAAAAAAAAA CA---------
A---------- -> AAAAAAAAAAA A----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------- -> AAAAAAAAAAA -----------
----------T -> AAAAAAAAAGT ----------C
---------T- -> AAAAAAAAGTT ---------C-
--------T-- -> TTTTTTTTTG